# Tiling
Through this action, the previous mosaic is trimmed according to the limits of the desired administrative area. The result of this process is a multispectral image of the perfectly bounded study area.

In [47]:
import gdal, ogr, osr
import os
import time

print('Libraries imported successfully')

In [ ]:
seg_start=time.time() #For checking ejecution time

### Get layer extent
Return the extent of layer with GPKG or ESRI Shapefile formats

 - <b>layer_path</b>: Path to the layer

In [48]:
def get_layerextent(layer_path):
     # check if it is gpgk or shp file
    longitud = len(layer_path.split("."))
    driver_name = layer_path.split(".")[longitud - 1]
    if driver_name == "gpkg":
        driver = ogr.GetDriverByName("GPKG")
    if driver_name == "shp":
        driver = ogr.GetDriverByName("ESRI Shapefile")
        
     # open and get the extent
    ds = driver.Open(layer_path)
    xmin, xmax, ymin, ymax = ds.GetLayer().GetExtent()
    extent = f"{xmin}, {ymin}, {xmax}, {ymax}"

    del ds
    raster_extent=extent

### Layer within raster
check if a layer is inside the raster

 - <b>raster_extent</b>: extent of the raster
 
 - <b>layer_geom</b>: geometry of the layer
 
 - <b>lesser_lextetn</b>: If True a smaller extent is evaluated

In [ ]:
def layer_within_raster(raster_extent, layer_geom, lesser_lextent=False):

    rxmin, rxmax, rymin, rymax = raster_extent
    lxmin, lxmax, lymin, lymax = layer_geom.GetEnvelope()

    if lesser_lextent:
        # Getting a smaller bounding box
        lxmin = lxmin + 100
        lxmax = lxmax - 100
        lymin = lymin + 100
        lymax = lymax - 100

    i = 0
    if lxmin >= rxmin:  # 1. upper left corner
        i += 1
    if lymax <= rymax:  # 2. upper right corner
        i += 1
    if lxmax <= rxmax:  # 3. lower right corner
        i += 1
    if lymin >= rymin:  # 4. lower left corner
        i += 1

    if i == 4:
        out = True
    else:
        out = False
    return (out)

### Naming convention
Creates naming based on the raster name and geometries: date_xmin-ymax_sentineltile_band

 - <b>raster_path</b>: path to raster file
 
 - <b>geometry</b>: geometry of the feature

In [ ]:
def naming_convention(raster_path, geometry):

    # xmin, xmax, ymin, ymax
    xmin, xmax, ymin, ymax = geometry.GetEnvelope()
    splitted = raster_path.split("/")
    len_splitted = len(splitted)
    name_tmp1 = splitted[len_splitted - 1]
    name = name_tmp1.split(".")[0]
    name_splitted = name.split("_")
    if len(name_splitted) < 3:
        outaname = f"{name}_{float(xmin)}-{float(ymax)}"
    else:
        sent_tile = name_splitted[0]
        band = name_splitted[2]
        date_tmp = name_splitted[1]
        date = date_tmp.split("T")[0]

        outaname = f"{date}_{float(xmin)}-{float(ymax)}_{sent_tile}_{band}"
    return (outaname)

### Reproject
Function to reproject an image to a desire espg

 - <b>image</b>: path to raster image
 
 - <b>output_folder</b>: output folder where the output image will be saved
 
 - <b>epsg_to</b>: coordinate epsg code to reproject into. 25830 by deafult.

In [49]:
def reproject(image, output_folder, epsg_to=25830):
    
    splitted = image.split("/")
    lenout = len(splitted)
    out_name = splitted[lenout-1]
    output = f"{output_folder}/reprojeted_{out_name}"

    dataset = gdal.Open(image)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(epsg_to)
    vrt_ds = gdal.AutoCreateWarpedVRT(dataset, None, srs.ExportToWkt(), gdal.GRA_NearestNeighbour)

    # cols = vrt_ds.RasterXSize
    # rows = vrt_ds.RasterYSize
    gdal.GetDriverByName("GTiff").CreateCopy(output, vrt_ds)

### Masking tiles
It creates tiles from a raster image based on a grid previously created

 - <b>layer_tiles</b>: path to grid
 
 - <b>raster_path</b>: path to raster
 
 - <b>output_folder</b>: path to output folder
  
 - <b>field</b>: Field with cut tiles with
  
 - <b>naming</b>: Apply naming rule (function define before)
  
 - <b>extent</b>: Cut with extent  
 
 - <b>lesser_lextent</b>: create an smaller extent 
 
 - <b>reproyectar</b>: If True, reprojection is applied
   
 - <b>epsg</b>: EPSG code of the srs to reproject into

In [ ]:
def masking_tiles(layer_tiles,
                  raster_path,
                  output_folder,
                  field="fid_id",
                  naming=True,
                  extent=False, #Falta desarrollar extent para .tif
                  lesser_lextent=False,
                  reproyectar=False,
                  epsg=25830
                  ):
    
    if os.path.exists(output_folder) is False:
        os.mkdir(output_folder)

    if reproyectar:
        raster_path2 = raster_path
        raster_path = reproject(raster_path, output_folder, epsg_to=epsg)
        print(raster_path)

    driver=ogr.GetDriverByName("GPKG")
    ds = driver.Open(layer_tiles)
    layer = ds.GetLayer()
    i=0
    
    print('Creating masking tiles files')
    for feature in layer: 
        i=i+1
        geom = feature.geometry()
        fid = feature.GetField(field)
        if naming:
            if reproyectar:
                out_name = naming_convention(raster_path2, geom)
            else:
                out_name = naming_convention(raster_path, geom)
        else:
            out_tmp = raster_path.split("/")
            out_tmp2 = out_tmp[len(out_tmp) - 1]
            out_name = out_tmp2.split(".")[0]

        output = f"{output_folder}/{out_name}.tif"
        print(f'Done successfully: {out_name}')
        
        if extent:
            raster_extent = get_layerextent(raster_path)
            #raster_extent = get_rasterextent(raster_path) ORIGINAL
            sepuede = layer_within_raster(raster_extent, geom, lesser_lextent=lesser_lextent)

            if sepuede:
                xmin, xmax, ymin, ymax = geom.GetEnvelope()
                lextent = [xmin, ymin, xmax, ymax]

                ds2 = gdal.Warp(output,
                                raster_path,
                                format="GTiff",
                                outputBounds=lextent)

                del ds2

        else:
            ds2 = gdal.Warp(output,
                            raster_path,
                            format="GTiff",
                            cutlineDSName=layer_tiles,
                            cutlineWhere=f"{field} = '{fid}'",
                            cropToCutline=True)
            del ds2

    layer.ResetReading()
    ds.FlushCache()

    del ds

Define path to files and execute

In [ ]:
tmp = '/home/jovyan/work/DATASET/'
layer_tiles=f'{tmp}Dataset_files/grid_comunidad_valenciana.gpkg'
raster_path=f'{tmp}Dataset_processing/Mask/comunidad_valenciana_mask.tif'
output_folder=f'{tmp}Dataset_processing/Tiling'

masking_tiles(layer_tiles, raster_path, output_folder)

In [2]:
print(f'Tiling finished successfully in {time.time() - seg_start} s')

NameError: name 'time' is not defined